In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
torch.__version__

In [ ]:
torch.cuda.is_available()

In [ ]:
# mean and std values need to be obtained from dataset that you are using here we are using commonly used referring ImageNet dataset

transform = transforms.Compose([ transforms.Resize(64),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                            std=[0.229, 0.224, 0.225] ) ])

train_data_path = "/home/vaadmin/workspace/projects/pytorch-projects/dataset/train" 
train_data = torchvision.datasets.ImageFolder(root=train_data_path, transform=transform)

test_data_path = "/home/vaadmin/workspace/projects/pytorch-projects/dataset/test"
test_data = torchvision.datasets.ImageFolder(root=test_data_path, transform=transform)

val_data_path = "/home/vaadmin/workspace/projects/pytorch-projects/dataset/valid"



In [ ]:
batch_size = 64
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle=True, num_workers=6)

test_data_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size, shuffle=True, num_workers=6)

In [ ]:
next(iter(train_data_loader))

print(train_data.classes)

In [ ]:
print(next(iter(test_data_loader)))
print(test_data.classes)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(train_data_loader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print([train_data.classes[val] for val in labels])

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class CNNet(nn.Module):

    def __init__(self, num_classes=10):
        super(CNNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3,64,kernel_size=11, stride=4,padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),)

        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, num_classes)
            )

    def forward(self, x):
        x = self.features(x)
        # print(f" feature shape: {x.shape}")
        x = self.avgpool(x)
        # print(f" avgpool shape: {x.shape}")
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        # print(f" classifier shape: {x.shape}")
        return x


mynet = CNNet()

In [ ]:
X = torch.rand(32, 3, 64, 64)
logits = mynet(X)
print(logits.shape)
# pred_probab = nn.Softmax(dim=1)(logits)
# y_pred = pred_probab.argmax(1)
# print(f"Predicted class: {y_pred}")

In [ ]:
import torch.optim as optim

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(mynet.parameters(), lr=0.001, momentum=0.9)

In [ ]:
size = len(train_data_loader.dataset)
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_data_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # forward + backward + optimize
        outputs = mynet(inputs)
        loss = loss_fn(outputs, labels)
        
        # zero the parameter gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print statistics
        if i % 100 == 0:
            loss, current = loss.item(), (i + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

print('Finished Training')

In [ ]:
# Test the model
correct = 0
total = 0
with torch.no_grad():
    for data in test_data_loader:
        inputs, labels = data
        outputs = mynet(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))